In [23]:
import tensorflow as tf
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Normalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
import numpy as np

In [44]:
model = Sequential([
    Dense(activation="sigmoid",units=3 ,name="layer1"),
    Dense(activation="sigmoid",units=1,name="layer2")
])

In [25]:
def load_coffee_data():
    """ Creates a coffee roasting data set.
        roasting duration: 12-15 minutes is best
        temperature range: 175-260C is best
    """
    rng = np.random.default_rng(2)
    X = rng.random(400).reshape(-1,2)
    X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
    X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
    Y = np.zeros(len(X))
    
    i=0
    for t,d in X:
        y = -3/(260-175)*t + 21
        if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
            Y[i] = 1
        else:
            Y[i] = 0
        i += 1

    return (X, Y.reshape(-1,1))

In [43]:
x,y = load_coffee_data()

In [45]:
norm_layer = Normalization(axis=-1)
norm_layer.adapt(x)
x_norm = norm_layer(x)

In [46]:
x_norm = np.tile(x_norm,(1000,1))
y = np.tile(y,(1000,1))

In [47]:
model.compile(loss=BinaryCrossentropy(),optimizer=Adam(learning_rate=0.01))

In [57]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 3)                 9         
                                                                 
 layer2 (Dense)              (None, 1)                 4         
                                                                 
Total params: 13 (52.00 Byte)
Trainable params: 13 (52.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [48]:
history = model.fit(x_norm,y,epochs=10)

Epoch 1/10


6250/6250 [==============================] - 3s 443us/step - loss: 0.2544
Epoch 2/10
6250/6250 [==============================] - 3s 445us/step - loss: 0.1365
Epoch 3/10
6250/6250 [==============================] - 3s 441us/step - loss: 0.1273
Epoch 4/10
6250/6250 [==============================] - 3s 433us/step - loss: 0.1189
Epoch 5/10
6250/6250 [==============================] - 3s 435us/step - loss: 0.1118
Epoch 6/10
6250/6250 [==============================] - 3s 442us/step - loss: 0.0640
Epoch 7/10
6250/6250 [==============================] - 3s 442us/step - loss: 0.0168
Epoch 8/10
6250/6250 [==============================] - 3s 436us/step - loss: 0.0108
Epoch 9/10
6250/6250 [==============================] - 3s 435us/step - loss: 0.0076
Epoch 10/10
6250/6250 [==============================] - 3s 438us/step - loss: 0.0055


In [53]:
X_test = np.array([
    [200,13.9],  # positive example
    [200,17]])   # negative example

In [54]:
X_test = norm_layer(X_test)

In [59]:
pred = model.predict(X_test)

1/1 [==============================] - 0s 9ms/step


In [60]:
for i in range(pred.shape[0]):
    if pred[i].item() >= 0.5:
        print("Good Coffee")
    else:
        print("Bad Coffee")

Good Coffee
Bad Coffee
